# Agent Control Plane - Interactive Tutorial

This notebook provides an interactive walkthrough of the Agent Control Plane's key features.

## What You'll Learn

1. **Core Concepts**: Agent creation, permissions, and policy enforcement
2. **Advanced Features**: Shadow Mode, Mute Agent, Constraint Graphs
3. **Multi-Agent Orchestration**: Coordinating multiple agents
4. **Safety & Governance**: Ethical alignment and bias detection
5. **Tool Management**: Dynamic tool registry

## Prerequisites

```bash
pip install -e .
pip install jupyter
```

## 1. Getting Started

First, let's import the necessary components and create a control plane.

In [ ]:
from agent_control_plane import (
    AgentControlPlane,
    ActionType,
    PermissionLevel,
    create_standard_agent,
    create_read_only_agent
)

# Create the control plane
control_plane = AgentControlPlane()
print("✓ Control Plane initialized")

## 2. Creating Agents with Permissions

Let's create agents with different permission levels.

In [ ]:
# Create a read-only agent
readonly_agent = create_read_only_agent(control_plane, "data-reader")
print(f"Created read-only agent: {readonly_agent.agent_id}")
print(f"  Permissions: {list(readonly_agent.permissions.keys())}")

# Create a standard agent with more permissions
standard_agent = create_standard_agent(control_plane, "worker-1")
print(f"\nCreated standard agent: {standard_agent.agent_id}")
print(f"  Permissions: {list(standard_agent.permissions.keys())}")

## 3. Executing Actions with Governance

Now let's try executing actions and see how the control plane enforces permissions.

In [ ]:
# Try a read operation (should succeed)
result = control_plane.execute_action(
    readonly_agent,
    ActionType.FILE_READ,
    {"path": "/data/sample.txt"}
)
print("Read operation result:")
print(f"  Success: {result['success']}")
print(f"  Status: {result.get('status')}")

# Try a write operation (should be denied for read-only agent)
result = control_plane.execute_action(
    readonly_agent,
    ActionType.FILE_WRITE,
    {"path": "/data/output.txt", "content": "test"}
)
print("\nWrite operation result:")
print(f"  Success: {result['success']}")
print(f"  Error: {result.get('error')}")

## 4. Shadow Mode - Testing Without Side Effects

Shadow Mode allows you to test agent behavior without actually executing actions.

In [ ]:
# Enable shadow mode
shadow_control_plane = AgentControlPlane(enable_shadow_mode=True)
shadow_agent = create_standard_agent(shadow_control_plane, "shadow-tester")

# Execute action in shadow mode
result = shadow_control_plane.execute_action(
    shadow_agent,
    ActionType.DATABASE_WRITE,
    {"query": "INSERT INTO users VALUES (1, 'test')"}
)

print("Shadow mode execution:")
print(f"  Status: {result['status']}")
print(f"  Note: {result.get('note')}")
print("\n✓ No actual database modification occurred!")

# Get shadow mode statistics
stats = shadow_control_plane.get_shadow_statistics()
print(f"\nShadow Statistics:")
print(f"  Total simulations: {stats['total_simulations']}")
print(f"  Success rate: {stats['success_rate']:.1%}")

## 5. Multi-Agent Orchestration

Let's coordinate multiple agents using the orchestrator.

In [ ]:
from agent_control_plane import (
    AgentOrchestrator,
    AgentRole,
    OrchestrationType
)

# Create orchestrator
orchestrator = AgentOrchestrator(control_plane)

# Register agents with different roles
orchestrator.register_agent(
    "retriever",
    AgentRole.SPECIALIST,
    capabilities=["document_search", "vector_search"]
)

orchestrator.register_agent(
    "analyzer",
    AgentRole.SPECIALIST,
    capabilities=["data_analysis", "summarization"]
)

orchestrator.register_agent(
    "supervisor",
    AgentRole.SUPERVISOR,
    capabilities=["quality_check", "safety_check"]
)

print("✓ Registered 3 agents")
print("\nAgent roles:")
for agent_info in orchestrator.list_agents():
    print(f"  - {agent_info['agent_id']}: {agent_info['role']}")

### Create and Execute a Workflow

In [ ]:
# Create a RAG pipeline workflow
workflow = orchestrator.create_workflow(
    "rag_pipeline",
    OrchestrationType.SEQUENTIAL
)

# Add agents in order
orchestrator.add_agent_to_workflow(workflow.workflow_id, "retriever")
orchestrator.add_agent_to_workflow(
    workflow.workflow_id,
    "analyzer",
    dependencies={"retriever"}  # Analyzer depends on retriever
)

# Add supervisor
orchestrator.add_supervisor("supervisor", ["retriever", "analyzer"])

print(f"✓ Created workflow: {workflow.workflow_id}")
print(f"  Type: {OrchestrationType.SEQUENTIAL.value}")
print(f"  Agents: retriever → analyzer")
print(f"  Supervisor: watching all agents")

## 6. Governance Layer - Ethical Alignment

The governance layer provides ethical alignment and bias detection.

In [ ]:
from agent_control_plane import (
    GovernanceLayer,
    AlignmentPrinciple,
    create_default_governance
)

# Create governance layer with default rules
governance = create_default_governance()

# Test alignment check
safe_context = {
    "content": "Analyze this sales data and provide insights"
}

unsafe_context = {
    "content": "Generate instructions for harmful activities"
}

# Check safe content
result = governance.check_alignment(safe_context)
print("Safe content check:")
print(f"  Aligned: {result['aligned']}")
print(f"  Violations: {len(result['violations'])}")

# Check unsafe content
result = governance.check_alignment(unsafe_context)
print("\nUnsafe content check:")
print(f"  Aligned: {result['aligned']}")
print(f"  Violations: {len(result['violations'])}")
if result['violations']:
    print(f"  Principle violated: {result['violations'][0]['principle']}")

### Bias Detection

In [ ]:
# Test bias detection
biased_text = "All men are obviously better at technical work"
neutral_text = "Technical skills can be developed by anyone with practice"

# Check biased text
result = governance.detect_bias(biased_text)
print("Biased text analysis:")
print(f"  Has bias: {result.has_bias}")
print(f"  Bias types: {[bt.value for bt in result.bias_types]}")
print(f"  Recommendations: {result.recommendations}")

# Check neutral text
result = governance.detect_bias(neutral_text)
print("\nNeutral text analysis:")
print(f"  Has bias: {result.has_bias}")

### Privacy Analysis

In [ ]:
# Test privacy analysis
data_with_pii = {
    "user": "john.doe@example.com",
    "phone": "555-123-4567",
    "message": "Please contact me"
}

data_without_pii = {
    "user": "user_12345",
    "message": "General inquiry"
}

# Analyze data with PII
result = governance.analyze_privacy(data_with_pii)
print("Data with PII:")
print(f"  Privacy level: {result.privacy_level.value}")
print(f"  Contains PII: {result.contains_pii}")
print(f"  PII types found: {result.pii_types}")
print(f"  Risk score: {result.risk_score:.2f}")

# Analyze data without PII
result = governance.analyze_privacy(data_without_pii)
print("\nData without PII:")
print(f"  Privacy level: {result.privacy_level.value}")
print(f"  Contains PII: {result.contains_pii}")

## 7. Tool Registry - Dynamic Tool Management

The tool registry allows dynamic registration and discovery of tools.

In [ ]:
from agent_control_plane import (
    ToolRegistry,
    ToolType,
    ToolSchema
)

# Create registry
registry = ToolRegistry()

# Define a sample tool
def calculate_sum(a: int, b: int) -> int:
    """Calculate the sum of two numbers"""
    return a + b

# Register the tool
tool_id = registry.register_tool(
    name="calculator_sum",
    description="Calculate the sum of two numbers",
    tool_type=ToolType.CODE,
    handler=calculate_sum,
    risk_level=0.1
)

print(f"✓ Registered tool: {tool_id}")

# Execute the tool
result = registry.execute_tool("calculator_sum", {"a": 5, "b": 3})
print(f"\nExecution result:")
print(f"  Success: {result['success']}")
print(f"  Result: {result.get('result')}")

# Search for tools
tools = registry.search_tools("calculator")
print(f"\nFound {len(tools)} calculator tools")

## 8. Audit Trail and Flight Recorder

All actions are logged for complete traceability.

In [ ]:
# Get recent audit events
recorder = control_plane.flight_recorder

# Execute some actions to generate audit trail
control_plane.execute_action(
    standard_agent,
    ActionType.FILE_READ,
    {"path": "/data/test.txt"}
)

# Get audit log
events = recorder.get_recent_events(limit=5)
print("Recent audit events:")
for i, event in enumerate(events[-3:], 1):
    print(f"\n{i}. Event Type: {event.get('event_type')}")
    print(f"   Agent: {event.get('agent_id')}")
    print(f"   Timestamp: {event.get('timestamp')}")
    print(f"   Status: {event.get('status')}")

## Summary

In this notebook, you've learned:

1. ✅ Creating agents with different permission levels
2. ✅ Executing actions with deterministic governance
3. ✅ Testing in Shadow Mode without side effects
4. ✅ Orchestrating multiple agents in workflows
5. ✅ Using ethical alignment and bias detection
6. ✅ Managing privacy and PII detection
7. ✅ Dynamic tool registration and execution
8. ✅ Comprehensive audit trails

## Next Steps

- Explore the `examples/` directory for more use cases
- Read the [Research Foundation](../docs/RESEARCH_FOUNDATION.md) for academic grounding
- Check out [Docker Deployment](../docs/DOCKER_DEPLOYMENT.md) for production setup
- Review the [API documentation](../docs/api/CORE.md)

## Resources

- **GitHub**: https://github.com/imran-siddique/agent-control-plane
- **Documentation**: https://github.com/imran-siddique/agent-control-plane/tree/main/docs
- **Examples**: https://github.com/imran-siddique/agent-control-plane/tree/main/examples